In [23]:
import numpy as np
import keras
import game_env


In [24]:
env = game_env.RPSGameEnv()
env.reset()

In [25]:
episodes = 10
for i in range(episodes):
    state = env.reset()
    done = False

    score = 0

    for i in range(50):
        action = env.action_space.sample()
        next_state, reward, done, info = env.step(action)
        score += reward

    print(f'Score: {score}')


Score: -20
Score: -4
Score: 8
Score: -4
Score: 6
Score: 2
Score: -4
Score: -4
Score: 22
Score: -4


In [26]:
def build_model(shape, actions):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(1,2,5)))
    model.add(keras.layers.Dense(24, activation='relu'))
    model.add(keras.layers.Dense(24, activation='relu'))
    model.add(keras.layers.Dense(actions, activation='linear'))
    model.compile(loss='mse', optimizer=keras.optimizers.Adam(learning_rate=0.001))
    return model

In [27]:
shape = env.observation_space.shape
actions = env.action_space.n
shape

(2, 5)

In [28]:
model = build_model(shape, actions)

In [29]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 10)                0         
                                                                 
 dense_6 (Dense)             (None, 24)                264       
                                                                 
 dense_7 (Dense)             (None, 24)                600       
                                                                 
 dense_8 (Dense)             (None, 3)                 75        
                                                                 
Total params: 939
Trainable params: 939
Non-trainable params: 0
_________________________________________________________________


In [30]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import EpsGreedyQPolicy

In [31]:
def build_agent(model, actions):
    policy = EpsGreedyQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, nb_actions=actions, memory=memory, nb_steps_warmup=10, target_model_update=1e-2, policy=policy)
    return dqn


In [32]:
dqn = build_agent(model, actions)
dqn.compile(keras.optimizers.Adam(learning_rate=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1, nb_max_episode_steps=100)

Training for 50000 steps ...


AssertionError: 